In [2]:
from wabs import *
from collections import *
class WA1(WhBase):
    
    def __init__(self):
        super().__init__()
        
    xpt_by_chatname = lambda self, chatname: "//div[@class='_21S-L']//span[@dir='auto' and contains(text(),'" + chatname + "')]//ancestor::div[@class='_8nE1Y']"
        
    def xpls(self, xpls):
        xp, store = xpls if type(xpls) is list else [xpls], defaultdict()
        for i in xp.copy():
            try: store[i] = self.driver.find_elements(By.XPATH, i)
            except: store[i] = 'error'
                
    def xpbulk(self, xpt):
        if type(xpt) is str: return self.xptext(xpt)
        elif type(xpt) is list: return [self.xptext(n) for n in xpt]
        elif type(xpt) is dict: 
            return {k : [self.xptext(n) for n in v] if type(v) is list else self.xptext(v) for k, v in xpt.items()}
        else: return None
        
    def xe(self, xpt, click=False, text=False):
        try:
            x = self.wait.until(EC.presence_of_element_located((By.XPATH, xpt)))
            if click == True: x.click()
            if text == False: return x
            else: return x.text
        except: return None
        
    def xes(self, xpt):
        try: return self.driver.find_elements(By.XPATH, xpt)
        except: return None
        
    def pane_xpt(self, index=None, parent = "(//div[@id='pane-side']//div[@class='_8nE1Y'])", chatname=None):
        if index is not None and parent is not None:
            bi = parent + "[" + str(index) + "]"
        elif parent is None and chatname is not None:
            bi = "//div[@id='pane-side']//div[@class='_21S-L']//span[@dir='auto' and contains(text(),'" + chatname + "')]//ancestor::" + parent
        else:
            bi = parent
        dc = {'chat_name' : bi + "//div[@class='_21S-L']//span[@dir='auto']",
              'last_text' : bi + "//span[@data-testid='last-msg-status']",
              'last_sender' : bi + "//div[@class='vQ0w7']//span[@dir='auto']",
              'last_msg_time' : bi + "/div[@class='y_sn4']/div[@class='Dvjym']",
              'new_msg_notif' : bi + "//div[@class='Dvjym']//span[@data-testid='icon-unread-count']"}
        return dc

    def pane_chat_info(self, store_dc = defaultdict(), bs="//div[@id='pane-side']//div[@class='_21S-L']//span[@dir='auto']"):
        'fetch available chats info from pane without scroll'
        ls = self.xes("//div[@id='pane-side']//div[@class='_21S-L']//span[@dir='auto']")
        for i in ls:
            bychat = self.xpt_by_chatname(i.text)
            dc = self.pane_xpt(parent=bychat)
            for k, v in dc.copy().items():
                q = self.xptext(v)
                dc[k] = q if q is not None else ''
            store_dc = update_key_value(store_dc, dc)
        else:
            return store_dc
    
    def get_unread_msg(self, chatname='Emergency SOC Group', store_dc = defaultdict(),msgbase="(//div[@id='main']//div[@data-testid='msg-container'])"):
        'fetch unread available msg inside group without scroll'
        chatxpt = self.xpt_by_chatname(chatname)
        dc = self.pane_xpt(parent=chatxpt)
        print('chatname: ', chatname, 'unread_sms: ',  self.xptext(dc['new_msg_notif']))
        v = self.xptext("//div[@id='main']//span[@data-testid='conversation-info-header-chat-title']")
        if v is None or v not in chatname:
            try: self.click(chatxpt)
            except: self.select_chat(chatname)
        avmsg = self.xes(msgbase)
        for n in range(len(avmsg)):
            dc = self.chat_msg('last()-' + str(n))
            store_dc = update_key_value(store_dc, dc)
        else:
            df = self.dict_to_df(store_dc, chatname)
            return store_dc, df
    
    def duplicate_none_chk(self, sdic, dc):
        dpcount, nonecount = 0, 0
        for k, v in dc.items():
            if v is None: nonecount = nonecount + 1
            else:
                if v in list(sdic.values()): 
                    dpcount = dpcount + 1
        else:
            if nonecount >=3: none_flag = True
            else: none_flag = False

            if dpcount>=3: dp_flag = True
            else: dp_flag = False
            return none_flag, dp_flag


    def watem_iter(self, method, start=1, end=50, step=1, sdict=defaultdict()):
        dc_is_none = 0
        for n in range(start, end, step):
            dc = getattr(self, method)(n)
            if dc is None and dc_is_none>3:
                print('return caller sdict & n')
                return sdict, n
            elif dc is not None:
                if len(sdict) == 0: sdict = dc
                else:
                    nf, dpf = self.duplicate_none_chk(sdict, dc)
                    if nf == False and dpf == False:
                        sdict = update_key_value(sdict, dc)
                        dc_is_none = 0
                        yield dc, n
                    else:
                        dc_is_none = dc_is_none + 1
            else:
                dc_is_none = dc_is_none + 1
                print('dc is None', dc_is_none)
        else:
            return sdict, n

In [14]:
x = WA1()
d = x.get_driver
x.reload

(10) WhatsApp
reload success, whatsapp ready


In [15]:
attr = ['accessible_name', 'aria_role', 'clear', 'click', 'find_element', 'find_elements',
 'get_attribute', 'get_dom_attribute', 'get_property', 'id', 'is_displayed', 'is_enabled','is_selected', 'location',
 'location_once_scrolled_into_view','parent','rect', 'screenshot', 'screenshot_as_base64', 'screenshot_as_png', 'send_keys', 'shadow_root', 'size',
 'submit','tag_name', 'text', 'value_of_css_property']

In [20]:
msgbs = "(//div[@id='main']//div[@data-testid='msg-container'])"
dic = {'sender' : ['sender'], 'text' : [msgbs + "//div[@class='_21Ahp']/span[1]/span"],
       'q_sender': [msgbs + "//div[@class='_3pMOs yKTUI']//div[1]/span"],
       'q_text' : [msgbs + "//div[@class='_3pMOs yKTUI']//div[2]/span"],
       'datetime' : ['dttm'], 'time': ['NA'],
       'chatname': ["//div[@id='main']//span[@data-testid='conversation-info-header-chat-title']"]}
    
if x.select_chat('Emergency SOC Group') == 1:
    ls = x.xes(msgbs)

chat body name:  Emergency SOC Group
chat selection successful


In [38]:
dir(ls[0])

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_execute',
 '_id',
 '_parent',
 '_upload',
 'accessible_name',
 'aria_role',
 'clear',
 'click',
 'find_element',
 'find_elements',
 'get_attribute',
 'get_dom_attribute',
 'get_property',
 'id',
 'is_displayed',
 'is_enabled',
 'is_selected',
 'location',
 'location_once_scrolled_into_view',
 'parent',
 'rect',
 'screenshot',
 'screenshot_as_base64',
 'screenshot_as_png',
 'send_keys',
 'shadow_root',
 'size',
 'submit',
 'tag_name',
 'text',
 'value_of_css_property']

In [40]:
ls[0]._id

'608c4bb3-b471-4be4-b5ad-5930417813ae'

In [ ]:
help(a.get_property)

In [104]:
a.find_element(By.XPATH, "div")

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x01006643]
	(No symbol) [0x00F9BE21]
	(No symbol) [0x00E9DA9D]
	(No symbol) [0x00EA09E4]
	(No symbol) [0x00EA08AD]
	(No symbol) [0x00EA0B30]
	(No symbol) [0x00ED0FAC]
	(No symbol) [0x00ED147B]
	(No symbol) [0x00EC64C1]
	(No symbol) [0x00EEFDC4]
	(No symbol) [0x00EC641F]
	(No symbol) [0x00EF00D4]
	(No symbol) [0x00F06B09]
	(No symbol) [0x00EEFB76]
	(No symbol) [0x00EC49C1]
	(No symbol) [0x00EC5E5D]
	GetHandleVerifier [0x0127A142+2497106]
	GetHandleVerifier [0x012A85D3+2686691]
	GetHandleVerifier [0x012ABB9C+2700460]
	GetHandleVerifier [0x010B3B10+635936]
	(No symbol) [0x00FA4A1F]
	(No symbol) [0x00FAA418]
	(No symbol) [0x00FAA505]
	(No symbol) [0x00FB508B]
	BaseThreadInitThunk [0x75A100F9+25]
	RtlGetAppContainerNamedObjectPath [0x77467BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77467B8E+238]


In [71]:
store_dc = defaultdict()
ls = x.xes("//div[@id='pane-side']//div[@class='_21S-L']//span[@dir='auto']")
for i in ls:
    bychat = x.xpt_by_chatname(i.text)
    dc = x.pane_xpt(parent=bychat)
    for k, v in dc.copy().items():
        q = x.xptext(v)
        dc[k] = q if q is not None else ''
    store_dc = update_key_value(store_dc, dc)
else:
    print(x.dict_to_df(store_dc))

None


In [80]:
WebElement user = driver.findElement(By.className("JournalDev"));

In [66]:
def forward(d, x, fwto='onami', msgxpt="(//div[@id='main']//div[@data-testid='msg-container'])[last()]"):
    a1 = x.xpelem(msgxpt)
    ActionChains(d).move_to_element(a1).perform()
    x.click(x.xpelem("(//div[@data-testid='msg-container'])[last()]//span[@data-testid='down-context']"))
    tm.sleep(1)
    x.click("//ul//li[contains(.,'Forward')]")
    tm.sleep(1)
    x.click("//div[@class='_2IUvV']//span[@data-testid='forward']")
    xpel  = x.xpelem("(//div[contains(@role,'textbox')])[1]")
    ActionChains(d).move_to_element(xpel).click().perform()
    tm.sleep(1)
    ActionChains(d).send_keys_to_element(xpel, fwto).perform()
    tm.sleep(2)
    x.click("(//div[@class='_199zF _3j691 _2N1Gm'])[1]")
    tm.sleep(1)
    x.click("//span[@data-testid='send']")
    print('success')

In [38]:
bs= "(//div[@id='main']//div[@data-testid='msg-container'])"        
pn = "(//div[@id='pane-side']//div[@class='_8nE1Y'])"
xp = lambda d, xpt : d.find_elements(By.XPATH, xpt)
ac = ActionChains(d)

In [41]:
x.pane_chat_info()

In [8]:
a1 = x.sidepane_scroll(a1)

In [9]:
a1 = x.sidepane_scroll(a1)

In [10]:
a1 = x.sidepane_scroll(a1)

In [11]:
a1 = x.sidepane_scroll(a1)

In [12]:
a1 = x.sidepane_scroll(a1)

In [13]:
def unread_message(self, initial = 0, scrolls=25):
    self.chats_unread_count = 0
    self.chats = {""}
    for i in range(0, scrolls):
        self._user()
        self.driver.execute_script("document.getElementById('pane-side').scrollTop={}".format(initial))
        initial += 300
    else:
        print('unread sms for groups')
        print(self.chats_unread)

In [31]:
def xp(driver, xptdict, fn):
    if type(xpt) is str: return fn(xpt)
    elif type(xpt) is list: return [fn(n) for n in xpt]
    elif type(xpt) is dict:
        return {k : [fn(driver, k, n) for n in v] if type(v) is list else fn(driver, k, v) for k, v in xpt.items()}
    else: print('error')
        
def fnx(dr, name, xpt):
    print(xpt)
    return '21'

In [34]:
pane_chat_info(x)

AttributeError: 'WA1' object has no attribute 'xes'